<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/NN_Sparse_1_3_Relu_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

from tensorflow.keras import models, layers, regularizers, callbacks
from sklearn.model_selection import train_test_split

# Bayesian Network Data Generation 500, ..., 20000 Samples (sparse)

In [105]:
# Function to generate CPDs for the sparse structure with 15 nodes influencing SP
def generate_cpds_sparse_15_total_nodes():
    # Generate random probabilities for the independent nodes
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()

    # Create sparse dependency structures for the nodes
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    irt_probs = np.random.rand(3)  # Independent node
    irt_probs /= irt_probs.sum()

    ms_given_ir_probs = np.random.rand(3, 3)
    ms_given_ir_probs /= ms_given_ir_probs.sum(axis=0, keepdims=True)

    geo_probs = np.random.rand(3)  # Independent node
    geo_probs /= geo_probs.sum()

    ue_given_ms_probs = np.random.rand(3, 3)
    ue_given_ms_probs /= ue_given_ms_probs.sum(axis=0, keepdims=True)

    gdp_given_ei_probs = np.random.rand(3, 3)
    gdp_given_ei_probs /= gdp_given_ei_probs.sum(axis=0, keepdims=True)

    inf_probs = np.random.rand(3)  # Independent node
    inf_probs /= inf_probs.sum()

    # New nodes with sparse dependencies
    irp_given_gdp_probs = np.random.rand(3, 3)
    irp_given_gdp_probs /= irp_given_gdp_probs.sum(axis=0, keepdims=True)

    inv_probs = np.random.rand(3)  # Independent node
    inv_probs /= inv_probs.sum()

    ci_given_irp_probs = np.random.rand(3, 3)
    ci_given_irp_probs /= ci_given_irp_probs.sum(axis=0, keepdims=True)

    # Adding new nodes: oil, fx, and irb
    oil_probs = np.random.rand(3)  # Independent node
    oil_probs /= oil_probs.sum()

    fx_given_ue_probs = np.random.rand(3, 3)
    fx_given_ue_probs /= fx_given_ue_probs.sum(axis=0, keepdims=True)

    irb_given_ci_probs = np.random.rand(3, 3)
    irb_given_ci_probs /= irb_given_ci_probs.sum(axis=0, keepdims=True)

    # SP depends on all 15 nodes with minimal connections
    sp_probs = np.random.rand(3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return (ir_probs, ei_given_ir_probs, irt_probs, ms_given_ir_probs,
            geo_probs, ue_given_ms_probs, gdp_given_ei_probs, inf_probs,
            irp_given_gdp_probs, inv_probs, ci_given_irp_probs, oil_probs,
            fx_given_ue_probs, irb_given_ci_probs, sp_probs)

# Function to generate and save samples with the sparse structure of 15 nodes total
def generate_and_save_samples_sparse_15_total_nodes(ir_probs, ei_probs, irt_probs, ms_probs, geo_probs, ue_probs, gdp_probs, inf_probs, irp_probs, inv_probs, ci_probs, oil_probs, fx_probs, irb_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample independent and dependent nodes based on the sparse structure
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]

        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]

        irt_state_idx = np.random.choice(3, p=irt_probs)
        irt_state = ['weak', 'moderate', 'strong'][irt_state_idx]

        ms_probs_given_ir = ms_probs[:, ir_state_idx]
        ms_state_idx = np.random.choice(3, p=ms_probs_given_ir)
        ms_state = ['low', 'medium', 'high'][ms_state_idx]

        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['urban', 'suburban', 'rural'][geo_state_idx]

        ue_probs_given_ms = ue_probs[:, ms_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_ms)
        ue_state = ['low', 'medium', 'high'][ue_state_idx]

        gdp_probs_given_ei = gdp_probs[:, ei_state_idx]
        gdp_state_idx = np.random.choice(3, p=gdp_probs_given_ei)
        gdp_state = ['low', 'medium', 'high'][gdp_state_idx]

        inf_state_idx = np.random.choice(3, p=inf_probs)
        inf_state = ['low', 'medium', 'high'][inf_state_idx]

        irp_probs_given_gdp = irp_probs[:, gdp_state_idx]
        irp_state_idx = np.random.choice(3, p=irp_probs_given_gdp)
        irp_state = ['low', 'medium', 'high'][irp_state_idx]

        inv_state_idx = np.random.choice(3, p=inv_probs)
        inv_state = ['low', 'medium', 'high'][inv_state_idx]

        ci_probs_given_irp = ci_probs[:, irp_state_idx]
        ci_state_idx = np.random.choice(3, p=ci_probs_given_irp)
        ci_state = ['low', 'medium', 'high'][ci_state_idx]

        oil_state_idx = np.random.choice(3, p=oil_probs)
        oil_state = ['low', 'medium', 'high'][oil_state_idx]

        fx_probs_given_ue = fx_probs[:, ue_state_idx]
        fx_state_idx = np.random.choice(3, p=fx_probs_given_ue)
        fx_state = ['low', 'medium', 'high'][fx_state_idx]

        irb_probs_given_ci = irb_probs[:, ci_state_idx]
        irb_state_idx = np.random.choice(3, p=irb_probs_given_ci)
        irb_state = ['low', 'medium', 'high'][irb_state_idx]

        # Calculate SP probability based on the state of each node (sparse dependency on each)
        sp_probs_given_all = sp_probs[:, ir_state_idx, ei_state_idx, irt_state_idx, ms_state_idx, geo_state_idx, ue_state_idx, gdp_state_idx, inf_state_idx, irp_state_idx, inv_state_idx, ci_state_idx, oil_state_idx, fx_state_idx, irb_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_all)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]

        # Append sample data to output list including probabilities for all nodes
        output_data.append({
            'IR_State': ir_state,
            'EI_State': ei_state,
            'IRT_State': irt_state,
            'MS_State': ms_state,
            'GEO_State': geo_state,
            'UE_State': ue_state,
            'GDP_State': gdp_state,
            'INF_State': inf_state,
            'IRP_State': irp_state,
            'INV_State': inv_state,
            'CI_State': ci_state,
            'OIL_State': oil_state,
            'FX_State': fx_state,
            'IRB_State': irb_state,
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_all]),
            'Chosen_SP_State': sp_state
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes
sample_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000]

for size in sample_sizes:
    (ir_probs, ei_probs, irt_probs, ms_probs, geo_probs, ue_probs, gdp_probs, inf_probs, irp_probs, inv_probs, ci_probs, oil_probs, fx_probs, irb_probs, sp_probs) = generate_cpds_sparse_15_total_nodes()
    generate_and_save_samples_sparse_15_total_nodes(ir_probs, ei_probs, irt_probs, ms_probs, geo_probs, ue_probs, gdp_probs, inf_probs, irp_probs, inv_probs, ci_probs, oil_probs, fx_probs, irb_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+------------+-------------+------------+-------------+------------+-------------+-------------+-------------+-------------+------------+-------------+------------+-------------+-------------------------------------------------+-------------------+
|    | IR_State   | EI_State   | IRT_State   | MS_State   | GEO_State   | UE_State   | GDP_State   | INF_State   | IRP_State   | INV_State   | CI_State   | OIL_State   | FX_State   | IRB_State   | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |
+====+============+============+=============+============+=============+============+=============+=============+=============+=============+============+=============+============+=============+=================================================+===================+
|  0 | high       | good       | weak        | low        | urban       | high       | medium      | low         | low         | medium      |

# NN & KL-Div

In [106]:
# Sample sizes to loop through
sample_sizes = [50, 100, 500, 1000, 5000, 10000, 15000, 20000]

# Define the Neural Network architecture with L2 regularization
def create_nn_model(hidden_layers=1, nodes_per_layer=3, l2_lambda=0.01):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=(14,)))  # Updated input shape to match the number of features

    # Hidden layers with L2 regularization and Dropout
    for layer_num in range(hidden_layers):
        model.add(layers.Dense(
            nodes_per_layer,
            activation='relu',
            kernel_regularizer=regularizers.l2(l2_lambda),  # L2 regularization
            name=f"hidden_layer_{layer_num + 1}"
        ))
        model.add(layers.Dropout(0.2))  # Dropout layer to reduce overfitting

    # Output layer (3 classes: decrease, stable, increase) with L2 regularization
    model.add(layers.Dense(
        3,
        activation='softmax',
        kernel_regularizer=regularizers.l2(l2_lambda),
        name="output_layer"
    ))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Prepare a dictionary to store extracted data for each sample size
extracted_data = {}

# Extract the required columns from all sample sizes first
for size in sample_sizes:
    outcomes_file = f'combined_probabilities_{size}.csv'
    df = pd.read_csv(outcomes_file)

    # Include new nodes in the required columns, removed duplicate 'INF_State'
    required_columns = ['IR_State', 'EI_State', 'IRT_State', 'MS_State', 'GEO_State',
                        'UE_State', 'GDP_State', 'INF_State', 'IRP_State', 'INV_State', 'CI_State','OIL_State', 'FX_State', 'IRB_State', 'Chosen_SP_State']
    df_extracted = df[required_columns]

    # Encode categorical variables for all states
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    irt_map = {'weak': 0, 'moderate': 1, 'strong': 2}
    ms_map = {'low': 0, 'medium': 1, 'high': 2}
    geo_map = {'urban': 0, 'suburban': 1, 'rural': 2}
    ue_probs_map = {'low': 0, 'medium': 1, 'high': 2}
    gdp_probs_map = {'low': 0, 'medium': 1, 'high': 2}
    inf_probs_map = {'low': 0, 'medium': 1, 'high': 2}
    irp_probs_map = {'low': 0, 'medium': 1, 'high': 2}
    inv_probs_map = {'low': 0, 'medium': 1, 'high': 2}
    ci_probs_map = {'low': 0, 'medium': 1, 'high': 2}
    oil_map = {'low': 0, 'medium': 1, 'high': 2}
    fx_map = {'low': 0, 'medium': 1, 'high': 2}
    irb_map = {'low': 0, 'medium': 1, 'high': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_extracted['IR_encoded'] = df_extracted['IR_State'].map(ir_map)
    df_extracted['EI_encoded'] = df_extracted['EI_State'].map(ei_map)
    df_extracted['IRT_encoded'] = df_extracted['IRT_State'].map(irt_map)
    df_extracted['MS_encoded'] = df_extracted['MS_State'].map(ms_map)
    df_extracted['GEO_encoded'] = df_extracted['GEO_State'].map(geo_map)
    df_extracted['UE_encoded'] = df_extracted['UE_State'].map(ue_probs_map)
    df_extracted['GDP_encoded'] = df_extracted['GDP_State'].map(gdp_probs_map)
    df_extracted['INF_encoded'] = df_extracted['INF_State'].map(inf_probs_map)
    df_extracted['IRP_encoded'] = df_extracted['IRP_State'].map(irp_probs_map)
    df_extracted['INV_encoded'] = df_extracted['INV_State'].map(inv_probs_map)
    df_extracted['CI_encoded'] = df_extracted['CI_State'].map(ci_probs_map)
    df_extracted['OIL_encoded'] = df_extracted['OIL_State'].map(oil_map)
    df_extracted['FX_encoded'] = df_extracted['FX_State'].map(fx_map)
    df_extracted['IRB_encoded'] = df_extracted['IRB_State'].map(irb_map)
    df_extracted['SP_encoded'] = df_extracted['Chosen_SP_State'].map(sp_map)

    extracted_data[size] = df_extracted

# Initialize list to store K-L divergence and standard deviation results
results = []
epsilon = 1e-10  # Small value for smoothing

for size in sample_sizes:
    df = extracted_data[size]

    # Features (all encoded columns) and labels (SP)
    X = df[['IR_encoded', 'EI_encoded', 'IRT_encoded', 'MS_encoded', 'GEO_encoded', 'UE_encoded',
            'GDP_encoded', 'INF_encoded', 'IRP_encoded', 'INV_encoded', 'CI_encoded', 'OIL_encoded', 'FX_encoded', 'IRB_encoded']]
    y = df['SP_encoded']

    # Split into training, validation, and test sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=True, random_state=42)

    # Create and train the Neural Network model
    nn_model = create_nn_model(hidden_layers=1, nodes_per_layer=3, l2_lambda=0.01)
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    nn_model.fit(X_train, y_train, epochs=25, batch_size=16, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

    # Evaluate model accuracy
    train_loss, train_accuracy = nn_model.evaluate(X_train, y_train, verbose=0)
    val_loss, val_accuracy = nn_model.evaluate(X_val, y_val, verbose=0)
    test_loss, test_accuracy = nn_model.evaluate(X_test, y_test, verbose=0)

    print(f"\nSample size: {size}")
    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")

    # Predict on test data
    predictions = nn_model.predict(X_test)
    predicted_classes = predictions.argmax(axis=1)

    # Calculate ground truth and predicted probabilities
    ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
    predicted_probabilities = pd.Series(predicted_classes).value_counts(normalize=True).sort_index()

    # Reindex both distributions and add smoothing
    all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
    ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
    predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

    # Calculate K-L divergence and standard deviation
    kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
    std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

    results.append({
        'Sample_Size': size,
        'K-L_Divergence': kl_divergence,
        'Standard_Deviation': std_dev
    })

    print(f"K-L Divergence: {kl_divergence:.4f}")
    print(f"Standard Deviation: {std_dev:.4f}")

    # Map integers back to the original SP labels
    sp_reverse_map = ['decrease', 'stable', 'increase']
    predicted_labels = [sp_reverse_map[label] for label in predicted_classes]

    # Create DataFrame for displaying nodes, predicted SP, and chosen SP
    result_df = pd.DataFrame({
        'IR_State': df['IR_State'].iloc[X_test.index],
        'EI_State': df['EI_State'].iloc[X_test.index],
        'IRT_State': df['IRT_State'].iloc[X_test.index],
        'MS_State': df['MS_State'].iloc[X_test.index],
        'GEO_State': df['GEO_State'].iloc[X_test.index],
        'UE_State': df['UE_State'].iloc[X_test.index],
        'GDP_State': df['GDP_State'].iloc[X_test.index],
        'INF_State': df['INF_State'].iloc[X_test.index],
        'IRP_State': df['IRP_State'].iloc[X_test.index],
        'INV_State': df['INV_State'].iloc[X_test.index],
        'CI_State': df['CI_State'].iloc[X_test.index],
        'OIL_State': df['OIL_State'].iloc[X_test.index],
        'FX_State': df['FX_State'].iloc[X_test.index],
        'IRB_State': df['IRB_State'].iloc[X_test.index],
        'Chosen_SP': df['Chosen_SP_State'].iloc[X_test.index],
        'Predicted_SP': predicted_labels
    })
    print(f"\nPredicted Results for {size} samples (First 10 rows):")
    print(result_df.head(10))

    # Save results for this sample size in a dedicated CSV
    result_df.to_csv(f'test_results_{size}.csv', index=False)

# Save only K-L and Standard Deviation results to a summary file
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_results_summary.csv', index=False)

print("\nAll K-L divergence and standard deviation results have been saved in 'kl_std_results_summary.csv'.")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 50
Training Accuracy: 0.2857
Validation Accuracy: 0.7143
Test Accuracy: 0.5000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
K-L Divergence: 10.4732
Standard Deviation: 0.3536

Predicted Results for 50 samples (First 10 rows):
   IR_State EI_State IRT_State MS_State GEO_State UE_State GDP_State  \
19     high     good  moderate      low     urban   medium    medium   
4      high  average  moderate      low     urban   medium    medium   
13     high     good  moderate      low  suburban     high    medium   
8      high     good    strong      low     urban   medium    medium   
48     high     good    strong     high  suburban      low    medium   
32     high     good  moderate      low     urban     high    medium   
30   medium  average      weak     high     urban   medium    medium   
39   medium  average  moderate     high     urban   medium       low   

   INF_State IRP_State INV_State CI_State OIL_State FX_State IRB_State  \
19    medium       low    medium      low   

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 100
Training Accuracy: 0.4143
Validation Accuracy: 0.2000
Test Accuracy: 0.2667
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
K-L Divergence: 15.8004
Standard Deviation: 0.5193

Predicted Results for 100 samples (First 10 rows):
   IR_State EI_State IRT_State MS_State GEO_State UE_State GDP_State  \
96      low     good  moderate   medium  suburban   medium       low   
4       low  average      weak   medium     rural   medium    medium   
42      low     poor  moderate      low  suburban   medium      high   
77      low  average  moderate      low     rural   medium    medium   
10   medium  average    strong   medium     rural     high       low   
0       low  average    strong      low     urban   medium      high   
9       low     poor  moderate      low     urban   medium       low   
69      low     good  moderate   medium     urban     high      high   
73      low  average  moderate   medium     rural      low    medium   
83      low     good      weak   medium     r

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


...


Sample size: 10000
Training Accuracy: 0.3380
Validation Accuracy: 0.3447
Test Accuracy: 0.3300
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
K-L Divergence: 14.3297
Standard Deviation: 0.4740

Predicted Results for 10000 samples (First 10 rows):
     IR_State EI_State IRT_State MS_State GEO_State UE_State GDP_State  \
2697   medium     poor      weak   medium     rural   medium       low   
6871      low     good    strong      low     rural   medium      high   
3487   medium     good    strong      low     rural   medium       low   
92     medium     good      weak   medium     rural      low       low   
9537      low     poor    strong      low     rural     high       low   
3205     high     poor      weak     high     rural      low      high   
6641   medium  average      weak     high     rural     high      high   
8909   medium     good    strong      low     urban     high       low   
2884   medium     poor      weak      low     urban   medium       low   
7173      low     po

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 15000
Training Accuracy: 0.3329
Validation Accuracy: 0.3489
Test Accuracy: 0.3284
71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
K-L Divergence: 14.3656
Standard Deviation: 0.4751

Predicted Results for 15000 samples (First 10 rows):
      IR_State EI_State IRT_State MS_State GEO_State UE_State GDP_State  \
8602       low     good      weak   medium     rural     high       low   
438     medium  average      weak      low     rural      low    medium   
8094      high     good  moderate   medium     urban     high    medium   
14355   medium     good      weak      low  suburban      low      high   
8581    medium     good  moderate      low     rural   medium    medium   
12358   medium  average      weak   medium  suburban     high       low   
511        low  average  moderate   medium     rural     high       low   
6594       low     good      weak     high  suburban      low      high   
5245       low     good  moderate     high  suburban   medium    medium   
5437    me

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Sample size: 20000
Training Accuracy: 0.3382
Validation Accuracy: 0.3343
Test Accuracy: 0.3390
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
K-L Divergence: 14.1217
Standard Deviation: 0.4674

Predicted Results for 20000 samples (First 10 rows):
      IR_State EI_State IRT_State MS_State GEO_State UE_State GDP_State  \
5348      high     good  moderate      low     urban      low       low   
339     medium  average      weak   medium     urban   medium    medium   
13591     high     good  moderate     high     urban     high       low   
8153    medium     good      weak      low     urban     high       low   
16345   medium  average  moderate      low     urban   medium       low   
16404   medium  average    strong      low     urban      low    medium   
17185   medium     good  moderate      low  suburban   medium       low   
5709    medium     poor      weak      low     urban   medium    medium   
13020   medium  average  moderate      low     urban      low    medium   
7763    me